<a href="https://colab.research.google.com/github/Hanad-Muqaddar/GoogleColabNotebooks/blob/master/TextClassificationFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  12.6M      0  0:00:06  0:00:06 --:--:-- 15.3M


In [3]:
!tar -xf aclImdb_v1.tar.gz

In [4]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [5]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [6]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [7]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [8]:
!rm -r aclImdb/train/unsup

In [9]:
import tensorflow as tf
import numpy as np

In [10]:
batch_size = 32
raw_train_dataset = tf.keras.utils.text_dataset_from_directory("/content/aclImdb/train", 
                                                              batch_size=batch_size, validation_split=0.2,
                                                               subset="training", seed=1337)
raw_validation_dataset = tf.keras.utils.text_dataset_from_directory("/content/aclImdb/train",
                                                                    batch_size=batch_size, validation_split=0.2,
                                                                    subset="validation", seed=1337)
raw_testing_dataset = tf.keras.utils.text_dataset_from_directory("/content/aclImdb/test", 
                                                                 batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [11]:
for text, lable in raw_train_dataset.take(1):
  for i in range(1):
    print(text.numpy()[i])
    print(lable.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

# Prepare the data

In [12]:
from tensorflow import keras
from keras.layers import TextVectorization
import string
import re

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

max_features = 20000
embedding_dim = 128
sequence_length = 500


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

text_ds = raw_train_dataset.map(lambda x, y: x)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [13]:
vectorize_layer.adapt(text_ds)

In [14]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [15]:
train_ds = raw_train_dataset.map(vectorize_text)
val_ds = raw_validation_dataset.map(vectorize_text)
test_ds = raw_validation_dataset.map(vectorize_text)

In [16]:
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

# Build a Model

In [17]:
from keras import layers

inputs = tf.keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

x = layers.Conv1D(128,7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128,7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPool1D()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
epochs = 3

model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
625/625 [==============================] - 82s 115ms/step - loss: 0.5350 - accuracy: 0.6834 - val_loss: 0.3100 - val_accuracy: 0.8678
Epoch 2/3
625/625 [==============================] - 5s 8ms/step - loss: 0.2348 - accuracy: 0.9089 - val_loss: 0.3128 - val_accuracy: 0.8830
Epoch 3/3
625/625 [==============================] - 5s 7ms/step - loss: 0.1222 - accuracy: 0.9563 - val_loss: 0.3636 - val_accuracy: 0.8760


In [19]:
model.evaluate(test_ds)

157/157 [==============================] - 1s 7ms/step - loss: 0.3636 - accuracy: 0.8760


[0.36363011598587036, 0.8759999871253967]